In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
path_val = "images/validation/"
path_train = "images/train/"

train_datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(path_train,
        target_size=(277, 267),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(path_val,
        target_size=(277, 267),
        batch_size=32,
        class_mode='categorical')



Found 914 images belonging to 8 classes.
Found 32 images belonging to 8 classes.


In [3]:
learning_rate = keras.callbacks.ReduceLROnPlateau(monitor='accuracy',factor=0.1,patience=2,min_lr=0.0001)

model = Sequential()
model.add(Conv2D(75, (3, 3), input_shape=(277, 267, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(50, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(25, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(25))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps= validation_generator.samples // validation_generator.batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

C:\Users\ahmet\AppData\Local\Temp\ipykernel_10760\2656475705.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "d:\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Python\Python310\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "d:\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\Python\Python310\lib\asyncio\base_events.py", line 595, in run_forever
      self._run_once()
 

In [ ]:
val_loss, val_acc = model.evaluate_generator(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Validation Loss: ", val_loss)
print("Validation Acc: ", val_acc)

C:\Users\ahmet\AppData\Local\Temp\ipykernel_12456\104031207.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  val_loss, val_acc = model.evaluate_generator(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)


Validation Loss:  -30518759424.0
Validation Acc:  0.125
